# QB Resilience Metric Engineering

The goal, in short, is to figure out factors like OL play (and injuries), QB # of historical snaps entering game, etc.

-rather than drive-based, game- and prior-game based
    
        -(or can agg previous drives' pass rate, really, using modified script)
                -can also do pressure over season, qb-induce-pressure tendencey
        -play-action tendency, box count faced tendency in past for qb?
                -best way to bolster PA indicator is past PA indications

-can see if we incorp. IPA stats and other stuff not quite working elsewhere

tempo: run success rate vs pass success rate?

combine regular pass rate, xpass?

guiding q: how does it help pred. pass?

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain
import nfl_data_py as nfl
from utility_db_25 import mark_columns, TrainML, build_catboost, optuna_call, test_ML, build_transformer, get_momentum_cols, create_momentum_index, get_final_features, get_motion_cols, motion_complexity_score

import player_play? player?

In [2]:
pbp_df = nfl.import_pbp_data([2022])

2022 done.
Downcasting floats.


In [3]:
pbp_df.head(1).T[:50]

,0
play_id,1.0
game_id,2022_01_BAL_NYJ
old_game_id_x,2022091107
home_team,NYJ
away_team,BAL
season_type,REG
week,1
posteam,None
posteam_type,None
defteam,None


Note: success rate captures sacks, etc. -- maybe game-long success rate (baked into xpass situational tho too?)

drive_yards_penalized -- game-long penalty stats for teams?

n_box seen in game? man/zone? (looks ur getting)
    -looks seen, can add weight disparity for current play, see how theyre playing u
    -game-long pass rates etc

In [4]:
pbp_df.head(1).T[370:]

,0
xpass,NaN
pass_oe,NaN
nflverse_game_id,2022_01_BAL_NYJ
old_game_id_y,2022091107
possession_team,
offense_formation,None
offense_personnel,None
defenders_in_box,NaN
defense_personnel,None
number_of_pass_rushers,NaN


In [5]:
ftn_df = nfl.import_ftn_data([2022])

Downcasting floats.


In [6]:
ftn_df.head(1).T

,0
ftn_game_id,5879
nflverse_game_id,2022_01_BUF_LA
season,2022
week,1
ftn_play_id,963918
nflverse_play_id,56
starting_hash,R
qb_location,S
n_offense_backfield,1
n_defense_box,6


Use qb fault sack? historical box rates players see? past PA rates for qb? hash?

is_qb_out_of_pocket?

# moree stuff

In [7]:
root_dir = os.getcwd()

In [8]:
train_data=pd.read_csv(os.path.join(root_dir, "data/train_data.csv"))

### todo: unpack Man_def, etc. value meaning

n_defense_box	



In [9]:
train_data.head(1).T[449:]

,0
qb_plays,2196.0
play_id,56
old_game_id_x,2022090800
xpass_situational,0.515357
weight_all_sum,2639
db_ct,5
mean_pairwise_dist,10.823594
mean_offset_db_orientation_abs,39.849639


In [10]:
root_dir = os.getcwd()

In [11]:
df_play = pd.read_csv(os.path.join(root_dir,'data/plays.csv')).sort_values(by=['gameId','possessionTeam','playId'])

In [13]:
df_play['pff_manZone'] = df_play['pff_manZone'].fillna('UNK')

In [14]:
df_play['is_man'] = 0
df_play.loc[df_play['pff_manZone'] == 'Man','is_man'] = 1

### Get rolling man/zone ratios

In [15]:
# get rolling man defense counts & play numbers
df_play = pd.concat([df_play,df_play.groupby(['gameId','possessionTeam']).agg(man_ct_game=('is_man','cumsum'),off_play=('is_man','cumcount'))],axis=1)
df_play['off_play']+=1

In [16]:
df_play['man_ratio_pre'] = df_play['man_ct_game']/df_play['off_play']
df_play['man_ratio'] = df_play.groupby(['gameId','possessionTeam']).man_ratio_pre.shift(1)
df_play['man_ratio'] = df_play['man_ratio'].fillna(.2)

In [17]:
df_play[['gameId','playId','possessionTeam','is_man','man_ct_game','off_play','man_ratio_pre','man_ratio']].head(15)

,gameId,playId,possessionTeam,is_man,man_ct_game,off_play,man_ratio_pre,man_ratio
13368,2022090800,56,BUF,0,0,1,0.000000,0.200000
2660,2022090800,80,BUF,0,0,2,0.000000,0.000000
7042,2022090800,101,BUF,0,0,3,0.000000,0.000000
2909,2022090800,122,BUF,0,0,4,0.000000,0.000000
10143,2022090800,167,BUF,0,0,5,0.000000,0.000000
5958,2022090800,191,BUF,0,0,6,0.000000,0.000000
16048,2022090800,212,BUF,1,1,7,0.142857,0.000000
13120,2022090800,236,BUF,0,1,8,0.125000,0.142857
7941,2022090800,529,BUF,0,1,9,0.111111,0.125000
4015,2022090800,550,BUF,0,1,10,0.100000,0.111111


In [18]:
train_data=pd.read_csv(os.path.join(root_dir, "data/train_data.csv")).sort_values(by=['gameId','possessionTeam','playId'])

In [19]:
[x for x in train_data if 'box' in x]

['n_defense_box']

In [20]:
train_data = pd.concat([train_data,train_data.groupby(['gameId','possessionTeam']).agg(sum_box=('n_defense_box','cumsum'),off_play=('pass','cumcount'))],axis=1)
train_data['off_play']+=1
train_data['mean_bc_pre'] = train_data['sum_box']/train_data['off_play']
train_data['mean_box_ct'] = train_data.groupby(['gameId','possessionTeam']).mean_bc_pre.shift(1)
train_data['mean_box_ct'] = train_data['mean_box_ct'].fillna(6)

In [21]:
train_data = train_data.merge(df_play[['gameId','playId','man_ratio']],how='left')

# TODO: get EWM versions/rolling means

intuition is game plan shifts over scope of game

pull from what, qbr munging?

In [22]:
train_data['box_ewm_pre'] = train_data.groupby(['gameId','possessionTeam'])['n_defense_box'].transform(lambda x: x.ewm(alpha=.1).mean())
train_data['box_ewm'] = train_data.groupby(['gameId','possessionTeam']).box_ewm_pre.shift(1)

In [23]:
train_data[['gameId','playId','possessionTeam','n_defense_box','sum_box','off_play','mean_bc_pre','mean_box_ct','box_ewm']].head(15)

,gameId,playId,possessionTeam,n_defense_box,sum_box,off_play,mean_bc_pre,mean_box_ct,box_ewm
0,2022090800,56,BUF,6.0,6.0,1,6.000000,6.000000,NaN
1,2022090800,80,BUF,6.0,12.0,2,6.000000,6.000000,6.000000
2,2022090800,101,BUF,7.0,19.0,3,6.333333,6.000000,6.000000
3,2022090800,122,BUF,6.0,25.0,4,6.250000,6.333333,6.369004
4,2022090800,167,BUF,5.0,30.0,5,6.000000,6.250000,6.261704
5,2022090800,191,BUF,6.0,36.0,6,6.000000,6.000000,5.953603
6,2022090800,212,BUF,5.0,41.0,7,5.857143,6.000000,5.963505
7,2022090800,236,BUF,7.0,48.0,8,6.000000,5.857143,5.778821
8,2022090800,529,BUF,7.0,55.0,9,6.111111,6.000000,5.993238
9,2022090800,550,BUF,6.0,61.0,10,6.100000,6.111111,6.157586


In [24]:
train_data[['man_ratio','mean_box_ct','box_ewm','pass']].corr()

,man_ratio,mean_box_ct,box_ewm,pass
man_ratio,1.000000,0.142508,0.134879,-0.009945
mean_box_ct,0.142508,1.000000,0.921295,-0.049736
box_ewm,0.134879,0.921295,1.000000,-0.088376
pass,-0.009945,-0.049736,-0.088376,1.000000


In [25]:
box_ewm_corrs = train_data.corr()['box_ewm'].sort_values(ascending=False)
weight_corrs = train_data.corr()['weight_all_sum'].sort_values(ascending=False)

In [26]:
box_ewm_corrs.head(25)

box_ewm                   1.000000
box_ewm_pre               0.935949
mean_box_ct               0.921295
mean_bc_pre               0.890796
n_defense_box             0.254748
time_remaining            0.160234
man_ratio                 0.134879
weight_all_sum            0.130069
s|max|T_2                 0.112179
o_standard|std|T_2        0.108824
o_standard|mean|QB_1      0.108347
o_standard|mean|T_2       0.104709
y_standard|mean|T_2       0.102222
dis|sum|T_2               0.100384
humidity                  0.099808
dir_standard|std|T_2      0.098326
y_standard|std|T_2        0.093006
dir_standard|mean|TE_2    0.089463
x_standard|mean|TE_2      0.088521
dir_standard|std|TE_2     0.087405
o_standard|mean|TE_2      0.085919
Cover-2_def               0.085909
yac_att                   0.085630
y_standard|mean|TE_2      0.084659
a|max|T_2                 0.083191
Name: box_ewm, dtype: float64

In [27]:
weight_corrs.head(25)

weight_all_sum                 1.000000
o_standard|mean|TE_2           0.404034
y_standard|mean|TE_2           0.399727
x_standard|mean|TE_2           0.391052
dir_standard|std|TE_2          0.389062
dir_standard|mean|TE_2         0.387612
s|max|TE_2                     0.351831
n_defense_box                  0.351236
a|max|TE_2                     0.347494
x_standard|std|TE_2            0.345018
o_standard|std|TE_2            0.319780
dis|sum|TE_2                   0.312823
y_standard|std|TE_2            0.250187
n_offense_backfield            0.244578
x_standard|std|QB_1            0.241402
x_standard|mean|TE_3           0.235657
x_standard|mean|FB_1           0.233519
o_standard|mean|TE_3           0.233065
y_standard|mean|FB_1           0.231933
y_standard|mean|TE_3           0.229422
offenseFormation_SINGLEBACK    0.227644
dir_standard|std|FB_1          0.227638
dir_standard|mean|TE_3         0.227236
o_standard|mean|FB_1           0.224957
box_ewm_pre                    0.224176


### todo (maybe separate nb): pressure nums
    -issue: this won't tell jack for run plays
    -chicken-egg
    -calc other ways to measure OL wins? ybc?
    -hard to quantify if OL beat (can point this out in future work section)

In [28]:
df_player_play = pd.read_csv(os.path.join(root_dir,'data/player_play.csv'))

In [29]:
df_player_play['quarterbackHit'].value_counts()

0    353313
1      1414
Name: quarterbackHit, dtype: int64

# Get weight data

In [30]:
df_players= pd.read_csv(os.path.join(root_dir,'data/players.csv'))

In [31]:
df_players.head(1).T

,0
nflId,25511
height,6-4
weight,225
birthDate,1977-08-03
collegeName,Michigan
position,QB
displayName,Tom Brady


In [32]:
df_players = pd.concat([df_players,df_players['height'].str.split('-',n=1,expand=True).rename(columns={0:'h_ft',1:'h_in_pre'})],axis=1)

In [33]:
df_players['height_inches'] = df_players['h_ft'].astype(int)*12 + df_players['h_in_pre'].astype(int)

In [34]:
df_players['bmi'] = df_players['weight'] /(df_players['height_inches']**2)

In [35]:
df_bmi = df_player_play[['gameId','playId','nflId']].merge(df_players[['nflId','bmi','height_inches','weight','position']])

In [36]:
df_bmi.head(1)

,gameId,playId,nflId,bmi,height_inches,weight,position
0,2022090800,56,35472,0.054815,77,325,G


In [37]:
df_bmi.position.value_counts()

CB     42239
WR     40940
T      32468
G      31238
OLB    23345
DE     22988
ILB    21921
DT     21461
TE     20996
FS     18235
SS     17441
C      17376
RB     16480
QB     16215
NT      5696
MLB     3636
FB      1643
LB       306
DB       103
Name: position, dtype: int64

note: can also check ol + TE? see how each corr

get front 7 bmi? inconsistency?

In [38]:
ol_df = df_bmi[df_bmi['position'].isin(['C','G','T'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_OL_weight','height_inches':'mean_OL_height','bmi':'mean_OL_bmi'})
dl_df = df_bmi[df_bmi['position'].isin(['DT','NT','DE'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_DL_weight','height_inches':'mean_DL_height','bmi':'mean_DL_bmi'})
lb_df = df_bmi[df_bmi['position'].isin(['LB','OLB','ILB'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_LB_weight','height_inches':'mean_LB_height','bmi':'mean_LB_bmi'})
cb_df = df_bmi[df_bmi['position'].isin(['CB'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_CB_weight','height_inches':'mean_CB_height','bmi':'mean_CB_bmi'})
wr_df = df_bmi[df_bmi['position'].isin(['WR'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_WR_weight','height_inches':'mean_WR_height','bmi':'mean_WR_bmi'})
te_df = df_bmi[df_bmi['position'].isin(['TE'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_TE_weight','height_inches':'mean_TE_height','bmi':'mean_TE_bmi'})

In [39]:
#df_play['isDropback']

In [40]:
df_play = df_play.merge(ol_df,how='left')
df_play = df_play.merge(dl_df,how='left')
df_play = df_play.merge(lb_df,how='left')
df_play = df_play.merge(cb_df,how='left')
df_play = df_play.merge(wr_df,how='left')
df_play = df_play.merge(te_df,how='left')

In [41]:
df_play['wr_cb_bmi_delta'] = df_play['mean_WR_bmi']-df_play['mean_CB_bmi']
df_play['ol_dl_bmi_delta'] = df_play['mean_OL_bmi']-df_play['mean_DL_bmi']
df_play['ol_box_delta'] = df_play['mean_OL_bmi']-((df_play['mean_DL_bmi'] + df_play['mean_LB_bmi']) /2)
df_play['ol_plus_box_delta'] = ((df_play['mean_OL_bmi']+df_play['mean_TE_bmi'])/2)-((df_play['mean_DL_bmi'] + df_play['mean_LB_bmi']) /2)
df_play['box_weight'] = (df_play['mean_DL_weight'] + df_play['mean_LB_weight']) /2
df_play['box_bmi'] = (df_play['mean_DL_bmi'] + df_play['mean_LB_bmi']) /2

In [42]:
df_play.columns[-28:]

Index(['man_ct_game', 'off_play', 'man_ratio_pre', 'man_ratio',
       'mean_OL_weight', 'mean_OL_height', 'mean_OL_bmi', 'mean_DL_weight',
       'mean_DL_height', 'mean_DL_bmi', 'mean_LB_weight', 'mean_LB_height',
       'mean_LB_bmi', 'mean_CB_weight', 'mean_CB_height', 'mean_CB_bmi',
       'mean_WR_weight', 'mean_WR_height', 'mean_WR_bmi', 'mean_TE_weight',
       'mean_TE_height', 'mean_TE_bmi', 'wr_cb_bmi_delta', 'ol_dl_bmi_delta',
       'ol_box_delta', 'ol_plus_box_delta', 'box_weight', 'box_bmi'],
      dtype='object')

In [43]:
train_data = train_data.merge(df_play[['gameId','playId']+list(df_play.columns[-28:])],how='left')

In [44]:
train_data[list(df_play.columns[-18:]) + ['pass']].corr()['pass']

mean_LB_weight       0.038807
mean_LB_height       0.031017
mean_LB_bmi          0.011371
mean_CB_weight      -0.041523
mean_CB_height      -0.062524
mean_CB_bmi          0.017419
mean_WR_weight      -0.033695
mean_WR_height      -0.043341
mean_WR_bmi          0.003785
mean_TE_weight      -0.078476
mean_TE_height      -0.037337
mean_TE_bmi         -0.046830
wr_cb_bmi_delta     -0.005681
ol_dl_bmi_delta      0.094342
ol_box_delta         0.063319
ol_plus_box_delta    0.048439
box_weight          -0.059949
box_bmi             -0.082858
pass                 1.000000
Name: pass, dtype: float64

In [45]:
pass_corrs_sorted = abs(train_data.corr()['pass']).sort_values(ascending=False)

# TODO: re-calc means perchance?

some composite means of means maybe best if you just calc sum over denom (e.g., OL + TE )

In [46]:
weight_sum_sorted = abs(train_data.corr()['weight_all_sum']).sort_values(ascending=False)

In [47]:
weight_sum_sorted['pass']

0.2945555321216303

In [48]:
motion_cols=get_motion_cols(train_data.columns)
momentum_cols=get_momentum_cols(train_data.columns)
train_data=create_momentum_index(train_data, momentum_cols)
#test_data=create_momentum_index(test_data, momentum_cols)

train_data=motion_complexity_score(train_data, motion_cols)
#test_data=motion_complexity_score(test_data, motion_cols)

In [49]:
final_features=['xpass_situational',  'QB_RB1_offset','off_xpass','n_offense_backfield',#'receiverAlignment_3x2' #'QBdffy_RB', 'QBdff_T', 
        'motion-momentum', 
        'neg_Formations', 'mean_pairwise_dist']

In [50]:
train_data['box_ewm_dl_weight'] = train_data['box_ewm']*train_data['mean_DL_weight']
train_data['box_ewm_dl_bmi'] = train_data['box_ewm']*train_data['mean_DL_bmi']
train_data['box_ewm_weight'] = train_data['box_ewm']*train_data['box_weight']
train_data['box_ewm_bmi'] = train_data['box_ewm']*train_data['box_bmi']

Initial takeaway is that rolling box EWM, paired with mean DL BMI, tells us 

In [51]:
train_data[final_features+['mean_DL_weight','box_ewm','box_ewm_dl_weight','box_ewm_weight',
                           'box_ewm_dl_bmi','box_ewm_bmi','pass']].corr()

,xpass_situational,QB_RB1_offset,off_xpass,n_offense_backfield,motion-momentum,neg_Formations,mean_pairwise_dist,mean_DL_weight,box_ewm,box_ewm_dl_weight,box_ewm_weight,box_ewm_dl_bmi,box_ewm_bmi,pass
xpass_situational,1.000000,0.092432,0.123280,-0.278399,0.055583,-0.459718,0.387330,-0.213839,-0.110936,-0.200752,-0.148524,-0.192436,-0.158509,0.488006
QB_RB1_offset,0.092432,1.000000,-0.040572,-0.210775,0.027452,-0.133448,0.115273,0.003599,-0.006314,-0.002552,0.001777,0.004018,0.004883,0.097237
off_xpass,0.123280,-0.040572,1.000000,-0.096844,0.053168,-0.157291,0.122287,-0.077070,-0.124522,-0.145759,-0.138472,-0.162620,-0.151396,0.162012
n_offense_backfield,-0.278399,-0.210775,-0.096844,1.000000,0.004917,0.341777,-0.321354,0.093784,0.071419,0.107142,0.086511,0.100851,0.088263,-0.312946
motion-momentum,0.055583,0.027452,0.053168,0.004917,1.000000,-0.062888,-0.003989,0.015374,-0.014821,-0.006762,-0.014796,-0.019931,-0.024243,0.237531
neg_Formations,-0.459718,-0.133448,-0.157291,0.341777,-0.062888,1.000000,-0.386887,0.133249,0.110688,0.160396,0.129762,0.163932,0.140113,-0.409078
mean_pairwise_dist,0.387330,0.115273,0.122287,-0.321354,-0.003989,-0.386887,1.000000,-0.114291,-0.157140,-0.187631,-0.163238,-0.181235,-0.165335,0.309695
mean_DL_weight,-0.213839,0.003599,-0.077070,0.093784,0.015374,0.133249,-0.114291,1.000000,0.031528,0.554262,0.399698,0.532743,0.347638,-0.104612
box_ewm,-0.110936,-0.006314,-0.124522,0.071419,-0.014821,0.110688,-0.157140,0.031528,1.000000,0.848517,0.910676,0.816798,0.916867,-0.088376
box_ewm_dl_weight,-0.200752,-0.002552,-0.145759,0.107142,-0.006762,0.160396,-0.187631,0.554262,0.848517,1.000000,0.971656,0.962315,0.949772,-0.128798


# Pressure EDA:

In [52]:
df_player_play['causedPressure'].value_counts()

False    350420
True       4307
Name: causedPressure, dtype: int64

In [53]:
#df_player_play.groupby(['gameId','playId']).agg().sum().reset_index()['causedPressure'].value_counts()

In [54]:
df_pp_cp = df_player_play.groupby(['gameId','playId']).agg(pressure_play=('causedPressure','any')).reset_index()

# TODO: rolling pressure in game (drive?)

In [55]:
df_play = df_play.merge(df_pp_cp,how='left')

In [56]:
df_play.sort_values(by=['gameId','possessionTeam','playId'],inplace=True)

In [57]:
df_play = pd.concat([df_play,df_play.groupby(['gameId','possessionTeam']).agg(pressure_ct=('pressure_play','cumsum'))],axis=1)

In [58]:
df_play['mean_pc_pre'] = df_play['pressure_ct']/df_play['off_play']
df_play['mean_pressure_ct'] = df_play.groupby(['gameId','possessionTeam']).mean_pc_pre.shift(1)
df_play['mean_pressure_ct'] = df_play['mean_pressure_ct'].fillna(0)

In [59]:
df_play[['gameId','playId','possessionTeam','pressure_play','pressure_ct','off_play','mean_pc_pre','mean_pressure_ct','isDropback']].head(12)

,gameId,playId,possessionTeam,pressure_play,pressure_ct,off_play,mean_pc_pre,mean_pressure_ct,isDropback
0,2022090800,56,BUF,False,0,1,0.000000,0.000000,True
1,2022090800,80,BUF,True,1,2,0.500000,0.000000,True
2,2022090800,101,BUF,False,1,3,0.333333,0.500000,False
3,2022090800,122,BUF,True,2,4,0.500000,0.333333,True
4,2022090800,167,BUF,False,2,5,0.400000,0.500000,True
5,2022090800,191,BUF,False,2,6,0.333333,0.400000,False
6,2022090800,212,BUF,False,2,7,0.285714,0.333333,True
7,2022090800,236,BUF,False,2,8,0.250000,0.285714,True
8,2022090800,529,BUF,False,2,9,0.222222,0.250000,False
9,2022090800,550,BUF,True,3,10,0.300000,0.222222,True


# TODO: combine with the line-box-count thing

In [60]:
df_play[['mean_pressure_ct','isDropback']].corr()

,mean_pressure_ct,isDropback
mean_pressure_ct,1.000000,0.028856
isDropback,0.028856,1.000000


In [61]:
train_data = train_data.merge(df_play[['gameId','playId','mean_pressure_ct']],how='left')

In [62]:
train_data['box_pressure']  = 5*train_data['box_ewm']+.05*train_data['mean_pressure_ct']	

In [81]:
train_data['OL_pc'] = train_data['mean_OL_bmi']*train_data['mean_pressure_ct']
train_data['DL_pc'] = train_data['mean_DL_bmi']*train_data['mean_pressure_ct']

In [82]:
train_data[['box_ewm_dl_bmi','box_ewm','box_pressure','mean_pressure_ct','OL_pc','DL_pc','pass']].corr()

,box_ewm_dl_bmi,box_ewm,box_pressure,mean_pressure_ct,OL_pc,DL_pc,pass
box_ewm_dl_bmi,1.000000,0.816798,0.816768,-0.162164,-0.163178,-0.108491,-0.130542
box_ewm,0.816798,1.000000,0.999997,-0.184349,-0.188041,-0.183528,-0.088376
box_pressure,0.816768,0.999997,1.000000,-0.182030,-0.185695,-0.181192,-0.088366
mean_pressure_ct,-0.162164,-0.184349,-0.182030,1.000000,0.999012,0.994687,0.025978
OL_pc,-0.163178,-0.188041,-0.185695,0.999012,1.000000,0.994076,0.024915
DL_pc,-0.108491,-0.183528,-0.181192,0.994687,0.994076,1.000000,0.015762
pass,-0.130542,-0.088376,-0.088366,0.025978,0.024915,0.015762,1.000000


In [65]:
abs(train_data.corr()['mean_pressure_ct']).sort_values(ascending=False).head(12)

mean_pressure_ct     1.000000
box_ewm              0.184349
box_pressure         0.182030
box_ewm_pre          0.176555
box_ewm_weight       0.175093
mean_bc_pre          0.174485
box_ewm_dl_weight    0.172591
box_ewm_bmi          0.172132
mean_box_ct          0.164844
playId               0.164247
play_id              0.164247
time_remaining       0.163668
Name: mean_pressure_ct, dtype: float64